In [1]:
import re
import pandas as pd
import numpy as np
import MySQLdb
import time
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing

In [2]:
start_timestamp = '1433088000' #1433088000 #1484870400
conn=MySQLdb.connect("54.65.19.253", "news", "newsnews", "news")
sql = "SELECT timestamp, tweet FROM trump WHERE timestamp > %s;"%(start_timestamp)
tStart = time.time()
trump_data = pd.read_sql_query(sql, conn)
sql = "SELECT timestamp, usdx FROM usdx WHERE timestamp > %s;"%(start_timestamp)
usdx_data = pd.read_sql_query(sql, conn)
tEnd = time.time()
conn.close()
print("Fetch data time: " +str(tEnd-tStart)+ "sec.")

Fetch data time: 0.45571112632751465sec.


In [3]:
trump_data.head()

,timestamp,tweet
0,1451491200,I would like to wish everyone A HAPPY AND HEAL...
1,1451491200,"HAPPY BIRTHDAY to my son, @DonaldJTrumpJr! Ver..."
2,1451491200,#MakeAmericaGreatAgain #Trump2016 https://t.co...
3,1451491200,"""@whispers34: Nevada Poll: Donald Trump 33%, T..."
4,1451491200,"""@stylin1188: Donald Trump is running the most..."


In [4]:
trump_data = trump_data.groupby('timestamp', as_index=False).agg(lambda tweet: ' '.join(tweet))
Y = usdx_data.as_matrix(columns=['usdx']).reshape(-1)
usdx_data['gradient'] = np.gradient(Y)

In [5]:
data = pd.merge(trump_data, usdx_data, on='timestamp')

In [6]:
data.head()
print(data.shape)

(474, 4)


In [7]:
Y = data.as_matrix(columns=['gradient']).reshape(-1)
offset = np.median(Y)
print(offset)
target_names = ['down','up']
Y[np.where(Y > 0)] = 1
Y[np.where(Y <= 0)] = 0
Y = Y.astype('int64')
f = np.bincount(Y)
print(f)
#print(1-((f[0]+f[1])/np.sum(f)))
#Y = shift(Y, -1, cval=1)

0.02105
[218 256]


In [8]:
vectorizer = TfidfVectorizer(min_df=4, ngram_range=(2, 4))
X = vectorizer.fit_transform(data['tweet'].tolist())
X

<474x2957 sparse matrix of type '<class 'numpy.float64'>'
	with 26428 stored elements in Compressed Sparse Row format>

In [9]:
X= hstack([X, X.power(2), X.power(3), X.power(4)])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
print(X_train.shape)
print(X_test.shape)

(379, 11828)
(95, 11828)


In [11]:
def create_model():
    model = NuSVC(kernel='rbf',
              nu=0.5,
              degree = 3,
              verbose=0)
    return model

In [12]:
cv = 5
model = create_model()
recall = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='recall')
precision = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='precision')
print('\nprecision:',np.mean(precision))
print('recall:', np.mean(recall))
#print('acc: %.4f'%((np.bincount(np.where(cross_val_predict(model, X, Y, cv=cv, n_jobs = -1)-Y != 0, 0, 1))/len(Y))[1]))
print(np.mean(cross_val_score(model, X, Y, cv=cv, n_jobs = -1)))


precision: 0.54651597682
recall: 0.937254901961
0.546452034341


In [13]:
skf = StratifiedKFold(n_splits=cv, shuffle=True)
for train_idx, test_idx in skf.split(X, Y):
    print ("Running Fold")
    model = create_model()
    model.fit(X.tocsr()[train_idx], Y[train_idx])
    print(confusion_matrix(y_test, model.predict(X_test), labels=[0,1]))
    print(classification_report(Y[test_idx], model.predict(X.toarray()[test_idx]), target_names=target_names))

Running Fold
[[24 22]
 [10 39]]
             precision    recall  f1-score   support

       down       0.54      0.45      0.49        44
         up       0.59      0.67      0.63        52

avg / total       0.57      0.57      0.57        96

Running Fold
[[18 28]
 [ 8 41]]
             precision    recall  f1-score   support

       down       0.52      0.32      0.39        44
         up       0.56      0.75      0.64        51

avg / total       0.54      0.55      0.53        95

Running Fold
[[12 34]
 [ 3 46]]
             precision    recall  f1-score   support

       down       0.58      0.16      0.25        44
         up       0.55      0.90      0.69        51

avg / total       0.57      0.56      0.48        95

Running Fold
[[13 33]
 [ 4 45]]
             precision    recall  f1-score   support

       down       0.62      0.19      0.29        43
         up       0.57      0.90      0.70        51

avg / total       0.59      0.57      0.51        94

Running Fold